In [1]:

# Barebones code to plot interactive correlation plot on web

In [8]:
def log(msg):
    with open("voila_debug_log.txt", "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")

# Optional: clear log at start of each session
with open("voila_debug_log.txt", "w") as f:
    f.write("Starting new Voila session...\n")


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from scipy.stats import pearsonr



In [7]:
# Load the saved CSV file back into a dataframe
merged_data = pd.read_csv(
    r"C:\Users\Rodriguez Lab\OneDrive - University of North Carolina at Chapel Hill\Desktop\Josh-Desktop\Marsh_Levee_Project\data-workup\merged_data_MAR_LOI_GS.csv",
    index_col=0)

In [9]:
# Automatically extract site names and variable types from column names
site_names = sorted(set(col.split('_')[0] for col in merged_data.columns if "_Year" in col))
variables = sorted(set(col.split('_')[1] for col in merged_data.columns if not col.endswith("Year")))


In [25]:
from plotly.offline import plot
from IPython.display import display, HTML
from scipy.stats import pearsonr

# Color-blind-friendly palette (Color Universal Design - CUD)
CUD_COLORS = {
    "LOI": "#E69F00",
    "MAR": "#56B4E9",
    "mean": "#009E73",
    "DBD": "#F0E442",
    "percent_clay": "#0072B2",
    "default": "#999999"
}

def log(msg):
    with open("voila_debug_log.txt", "a", encoding="utf-8") as f:
        f.write(str(msg) + "\n")

# Optional: clear the log at the start of the session
with open("voila_debug_log.txt", "w") as f:
    f.write("Starting new Voila session...\n")

def update_plot(site, year_range, xvar, yvar):
    try:
        log(f"[DEBUG] Running update_plot with: site={site}, year_range={year_range}, var1={xvar}, var2={yvar}")

        year_col = f'{site}_Year'
        x_col = f'{site}_{xvar}'
        y_col = f'{site}_{yvar}'

        log(f"[DEBUG] Columns being used: {year_col}, {x_col}, {y_col}")
        df = merged_data[[year_col, x_col, y_col]].dropna()
        df = df.rename(columns={year_col: 'Year', x_col: 'X', y_col: 'Y'})

        df_full = df.copy()
        df_zoom = df[(df['Year'] >= year_range[0]) & (df['Year'] <= year_range[1])]
        log(f"[DEBUG] Zoomed DataFrame length: {len(df_zoom)}")

        if len(df_zoom) >= 2 and df_zoom['X'].nunique() > 1 and df_zoom['Y'].nunique() > 1:
            r, p = pearsonr(df_zoom['X'], df_zoom['Y'])
            corr_label = f"Pearson r = {r:.2f}, p = {p:.2g}"
        else:
            corr_label = "Correlation not valid in this window"

        log(f"[DEBUG] corr_label: {corr_label}")
        log("[DEBUG] Plotting...")

        color1 = CUD_COLORS.get(xvar, CUD_COLORS["default"])
        color2 = CUD_COLORS.get(yvar, CUD_COLORS["default"])

        fig = go.Figure()

        # Full background
        fig.add_trace(go.Scatter(
            x=df_full['Year'], y=df_full['X'],
            mode='lines+markers', name=f'{xvar} (all)',
            line=dict(color=color1, dash='dot'),
            yaxis='y'
        ))

        fig.add_trace(go.Scatter(
            x=df_full['Year'], y=df_full['Y'],
            mode='lines+markers', name=f'{yvar} (all)',
            line=dict(color=color2, dash='dot'),
            yaxis='y2'
        ))

        # Zoomed window
        fig.add_trace(go.Scatter(
            x=df_zoom['Year'], y=df_zoom['X'],
            mode='lines+markers', name=f'{xvar} (zoom)',
            line=dict(color=color1, width=4),
            yaxis='y'
        ))

        fig.add_trace(go.Scatter(
            x=df_zoom['Year'], y=df_zoom['Y'],
            mode='lines+markers', name=f'{yvar} (zoom)',
            line=dict(color=color2, width=4),
            yaxis='y2'
        ))

        fig.update_layout(
            title=dict(
                text=f"{site}: Variable 1 = {xvar}, Variable 2 = {yvar} ({year_range[0]}–{year_range[1]})",
                x=0.5,
                xanchor='center'
            ),
            annotations=[
                dict(
                    text=corr_label,
                    xref='paper', yref='paper',
                    x=0.5, y=1.07,
                    showarrow=False,
                    font=dict(size=12)
                )
            ],
            xaxis=dict(title='Year'),
            yaxis=dict(
                title=dict(text=f'Variable 1: {xvar}', font=dict(color=color1)),
                tickfont=dict(color=color1)
            ),
            yaxis2=dict(
                title=dict(text=f'Variable 2: {yvar}', font=dict(color=color2)),
                tickfont=dict(color=color2),
                overlaying='y',
                side='right',
                showgrid=False  # hide conflicting gridlines
            ),

            legend=dict(
                orientation="h",
                yanchor="top",
                y=-0.25,
                xanchor="center",
                x=0.5
            ),
            height=500,
        )

        display(HTML(plot(fig, include_plotlyjs=True, output_type='div')))

    except Exception as e:
        log(f"[ERROR] Exception in update_plot: {e}")


In [26]:
site_selector = widgets.Dropdown(
    options=site_names,
    value=site_names[0],
    description='Site:'
)

year_slider = widgets.IntRangeSlider(
    value=[1980, 2000],
    min=1950,
    max=2022,
    step=1,
    description='Year Range:',
    continuous_update=False
)

x_selector = widgets.Dropdown(
    options=variables,
    value='MAR',
    description='X Variable:'
)

y_selector = widgets.Dropdown(
    options=variables,
    value='LOI',
    description='Y Variable:'
)

widgets.interact(
    update_plot,
    site=site_selector,
    year_range=year_slider,
    xvar=x_selector,
    yvar=y_selector
)


interactive(children=(Dropdown(description='Site:', options=('OC-21-01', 'OC-21-02', 'OC-21-03'), value='OC-21…

<function __main__.update_plot(site, year_range, xvar, yvar)>